In [427]:
from qiskit import quantum_info as qi
from typing import List
from itertools import product
import numpy as np
import scipy as sc
import qiskit.circuit.library as lib
import qiskit

Pauli = List[int]

np.set_printoptions(precision = 3, suppress=True)
from numba import vectorize

ST_IDX = 2 #starting index for binary strings
n = 2 #global number of qubits
d = 2**n #global size of hilbert space
P_1 = [np.identity(2), qi.Pauli('X').to_matrix(), qi.Pauli('Z').to_matrix()] 
pauli_names = ['I', 'X', 'Z', 'Y']
B_gates = [np.identity(2), lib.HGate().to_matrix(), np.identity(2), lib.HGate().to_matrix() @ lib.SGate().to_matrix()]

In [387]:
#return the name associated with a pauli binary string
def p_name(a: Pauli):
    p_name = ""
    for i in range(n):
        p_name += pauli_names[a[i]+a[i+n]*2]
    return p_name

#binary representation of pauli operator
def P(a: Pauli):
    ret_op = [1]
    for i in range(n):
        op = np.identity(2)
        op = op @ P_1[a[i]]
        op = op @ P_1[a[i+n]*2]
        op = op * 1j ** np.dot(a[n:], a[:n])
        ret_op = np.kron(ret_op,op)
    return ret_op

#define the binary symplectic product
def bs_prod(a : Pauli, b : Pauli):
    tot = 0
    for i in range(n):
        tot += a[i]*b[i+n]+a[i+n]*b[i]
    return tot % 2

def order_A(G : List[Pauli]):
    return 2**d/(d**2-len(G))

In [417]:
generators = [[1,1,0,1],[1,1,1,0]]
print([p_name(p) for p in generators])

['XY', 'YX']


In [402]:
def stabilizer_state(S, a):
    op = np.identity(d)
    for i in range(len(S)):
        sgn = (-1)**bs_prod(S[i], a)
        proj = .5*(np.identity(d)-np.multiply(sgn,P(S[i])))
        op = op @ proj
    return op

def stabilizer_measurement(S, a):
    p = S[0]
    B = np.ones(1)
    for i in range(n):
        print(p[i]+p[i+n])
        B = np.kron(B, B_gates[p[i]+p[i+n]*2])
    return B

In [403]:
stab = stabilizer_state(generators, [0,0,1,0])

In [425]:
vals, vecs = np.linalg.eig(stab)
proj = [vecs[i] for i in range(len(vecs)) if not np.abs(vals[i])<1e-16][0]

In [433]:
qc = qiskit.QuantumCircuit(2,2)
qc.initialize(proj)

In [429]:
qc.draw()

┌────────────────────────────────────────────┐
q_0: ┤0                                           ├
     │  Initialize(0.28868,-0.5,0.57735,-0.57735) │
q_1: ┤1                                           ├
     └────────────────────────────────────────────┘
c: 2/══════════════════════════════════════════════